# H5T0_CNN: implement a 2D CNN using PyTorch for image classification

### create your "signature":
#### Run the cell but do NOT modify the cell
#### The signature needs to be displayed right below the cell in this file

In [3]:
import platform, socket, getpass
from time import time
from datetime import datetime
import random
signature_for_cnn=[socket.gethostbyname(socket.gethostname()),
                   getpass.getuser(),
                   datetime.now(),
                   random.random()]
print(signature_for_cnn) 

['192.168.50.217', 'sloanatkins', datetime.datetime(2025, 12, 9, 15, 10, 36, 453819), 0.9452185987227159]


In [4]:
#sample code to save model and signature
#torch.save({"model":model.state_dict(),
#            "signature":signature_for_cnn},
#            "CNN_best.pt") 

### This is a complete application of image classification using a 2D CNN
steps: \
define a 2D CNN \
define a function for training the CNN \
define a function for testing the CNN \
define a function for saving the model and your `signature_for_cnn` \
load data \
train the CNN in a for loop using the training set and the validation set\
evalaute the best model on the test set\
display accuracy and confusion matrix for each of the three sets: train, validation, and test sets
### you may use the code in lecture nodes
### write your code below this line

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import torch.serialization

torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# LOAD DATA
# ---------------------------
data = torch.load("H5T0_train_val_test_data.pt", weights_only=False)

X_train = data["X_train"]
y_train = data["Y_train"]
X_val   = data["X_val"]
y_val   = data["Y_val"]
X_test  = data["X_test"]
y_test  = data["Y_test"]

# ---------------------------
# RESHAPE + CONVERT TO TENSORS
# ---------------------------
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype=torch.long)

X_val   = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
y_val   = torch.tensor(y_val, dtype=torch.long)

X_test  = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test  = torch.tensor(y_test, dtype=torch.long)

# ---------------------------
# DATA LOADERS
# ---------------------------
train_ds = TensorDataset(X_train, y_train)
val_ds   = TensorDataset(X_val, y_val)
test_ds  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False)

# ---------------------------
# DEFINE CNN MODEL
# ---------------------------
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),   # -> (32, 14, 14)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),   # -> (64, 7, 7)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# ---------------------------
# TRAIN ONE EPOCH
# ---------------------------
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(X)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# ---------------------------
# EVALUATION FUNCTION
# ---------------------------
def evaluate(model, loader):
    model.eval()
    preds_list = []
    labels_list = []

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            out = model(X)
            preds = torch.argmax(out, dim=1).cpu()
            preds_list.append(preds)
            labels_list.append(y)

    preds = torch.cat(preds_list)
    labels = torch.cat(labels_list)
    acc = accuracy_score(labels, preds)
    cm = confusion_matrix(labels, preds)
    return acc, cm


# ---------------------------
# TRAIN LOOP — TRACK BEST MODEL
# ---------------------------
best_val_acc = -1
best_state = None
num_epochs = 20

for epoch in range(num_epochs):
    loss = train_one_epoch(model, train_loader)
    train_acc, _ = evaluate(model, train_loader)
    val_acc, _   = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}/{num_epochs} | Loss={loss:.4f} | Train Acc={train_acc:.4f} | Val Acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = model.state_dict().copy()

print("\nBest validation accuracy =", best_val_acc)

# ---------------------------
# SAVE BEST MODEL
# ---------------------------
torch.save({
    "model": best_state,
    "signature": signature_for_cnn
}, "CNN_best.pt")

print("Saved best model as CNN_best.pt")

# ---------------------------
# LOAD BEST MODEL
# ---------------------------
best_model = CNN().to(device)
best_model.load_state_dict(best_state)

# ---------------------------
# FINAL EVALUATION
# ---------------------------
train_acc, train_cm = evaluate(best_model, train_loader)
val_acc, val_cm = evaluate(best_model, val_loader)
test_acc, test_cm = evaluate(best_model, test_loader)

print("\n=== FINAL ACCURACIES ===")
print("Train:", train_acc)
print("Val:", val_acc)
print("Test:", test_acc)

print("\n=== CONFUSION MATRICES ===")
print("Train:\n", train_cm)
print("\nVal:\n", val_cm)
print("\nTest:\n", test_cm)


/var/folders/fb/bmnkg3m57kb_ck5_xxc75zfh0000gn/T/ipykernel_47740/4284943603.py:9: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])


Using device: cpu
Epoch 1/20 | Loss=1.1413 | Train Acc=0.3130 | Val Acc=0.3190
Epoch 2/20 | Loss=0.5540 | Train Acc=0.7390 | Val Acc=0.7020
Epoch 3/20 | Loss=0.4320 | Train Acc=0.8370 | Val Acc=0.7610
Epoch 4/20 | Loss=0.3040 | Train Acc=0.9150 | Val Acc=0.8170
Epoch 5/20 | Loss=0.2613 | Train Acc=0.9590 | Val Acc=0.8260
Epoch 6/20 | Loss=0.1992 | Train Acc=0.9690 | Val Acc=0.8310
Epoch 7/20 | Loss=0.1388 | Train Acc=0.9840 | Val Acc=0.8150
Epoch 8/20 | Loss=0.1160 | Train Acc=0.9960 | Val Acc=0.8230
Epoch 9/20 | Loss=0.0966 | Train Acc=0.9950 | Val Acc=0.8340
Epoch 10/20 | Loss=0.0668 | Train Acc=0.9960 | Val Acc=0.8450
Epoch 11/20 | Loss=0.0570 | Train Acc=0.9920 | Val Acc=0.8320
Epoch 12/20 | Loss=0.0454 | Train Acc=1.0000 | Val Acc=0.8460
Epoch 13/20 | Loss=0.0257 | Train Acc=0.9990 | Val Acc=0.8380
Epoch 14/20 | Loss=0.0299 | Train Acc=0.9980 | Val Acc=0.8330
Epoch 15/20 | Loss=0.0266 | Train Acc=1.0000 | Val Acc=0.8510
Epoch 16/20 | Loss=0.0305 | Train Acc=0.9960 | Val Acc=0.8470